In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import chardet
import re


In [2]:
df = pd.read_csv('tennis_atp-master/atp_matches_2023.csv')

In [3]:
df.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2023-9900,United Cup,Hard,18,A,20230102,300,126203,3.0,NaN,...,62.0,47.0,15.0,12.0,9.0,9.0,9.0,3355.0,16.0,2375.0
1,2023-9900,United Cup,Hard,18,A,20230102,299,126207,NaN,NaN,...,12.0,8.0,3.0,4.0,1.0,3.0,19.0,2000.0,23.0,1865.0
2,2023-9900,United Cup,Hard,18,A,20230102,296,126203,3.0,NaN,...,62.0,51.0,7.0,12.0,2.0,2.0,9.0,3355.0,10.0,2905.0
3,2023-9900,United Cup,Hard,18,A,20230102,295,126207,NaN,NaN,...,41.0,26.0,12.0,9.0,6.0,9.0,19.0,2000.0,245.0,220.0
4,2023-9900,United Cup,Hard,18,A,20230102,292,126774,1.0,NaN,...,58.0,48.0,18.0,16.0,1.0,2.0,4.0,5550.0,16.0,2375.0


In [4]:
pbp = pd.read_csv('tennis_slam_pointbypoint-master/2023-wimbledon-points.csv')

In [5]:
pbp['match_id'].unique()

array(['2023-wimbledon-1101', '2023-wimbledon-1102',
       '2023-wimbledon-1103', '2023-wimbledon-1104',
       '2023-wimbledon-1105', '2023-wimbledon-1106',
       '2023-wimbledon-1107', '2023-wimbledon-1108',
       '2023-wimbledon-1109', '2023-wimbledon-1110',
       '2023-wimbledon-1111', '2023-wimbledon-1112',
       '2023-wimbledon-1113', '2023-wimbledon-1114',
       '2023-wimbledon-1115', '2023-wimbledon-1116',
       '2023-wimbledon-1117', '2023-wimbledon-1118',
       '2023-wimbledon-1119', '2023-wimbledon-1120',
       '2023-wimbledon-1121', '2023-wimbledon-1122',
       '2023-wimbledon-1123', '2023-wimbledon-1124',
       '2023-wimbledon-1125', '2023-wimbledon-1126',
       '2023-wimbledon-1127', '2023-wimbledon-1128',
       '2023-wimbledon-1129', '2023-wimbledon-1130',
       '2023-wimbledon-1131', '2023-wimbledon-1132',
       '2023-wimbledon-1133', '2023-wimbledon-1134',
       '2023-wimbledon-1135', '2023-wimbledon-1136',
       '2023-wimbledon-1137', '2023-wimbledon-

In [6]:
charting = pd.read_csv('tennis_MatchChartingProject-master/charting-m-points-2020s.csv', encoding= 'latin1', low_memory=False)

In [7]:
charting['rallyCount'] = pd.to_numeric(charting['rallyCount'], errors='coerce')
charting['rallyCount'].mean()

4.145064043563409

In [8]:
charting.iloc[65]['1st'] 

'4b2d#'

In [9]:
partidos = pd.Series(charting['match_id'].unique())
partidos.str.contains('Del Potro').sum()

0

# Rally tokenization

In [10]:
# Quito informacion adicional que no aporta informacion relevante
charting['1st'] = charting['1st'].str.replace(r'[c\-=\+;\^]', '', regex = True)
charting['2nd'] = charting['2nd'].str.replace(r'[c\-=\+;\^]', '', regex = True)

#2da manera

In [11]:
import pandas as pd
import re

errores = r'[ndxwg!e+]' 

# Parsing function (customize as needed)
def parse_sequence(sequence):
    result = []
    pos = 0  # Initialize position

    # Extract the serve data
    serve_pattern = rf'^([456]{errores}?)'
    match = re.match(serve_pattern, sequence)
    if match:
        serve = match.group(1)
        result.append(serve)
        pos = match.end()

    # Extract shot 1
    shot1_pattern = fr'([a-zA-Z]\d[789]?{errores}?)'
    match = re.match(shot1_pattern, sequence[pos:])
    if match:
        shot1 = match.group(1)
        result.append(shot1)
        pos += match.end()

    # Extract shots 2 to n
    shots_pattern = rf'([a-zA-Z]\d{errores}?)'
    shots_matches = re.finditer(shots_pattern, sequence[pos:])
    for m in shots_matches:
        result.append(m.group(1))
    pos += m.end() if 'm' in locals() else 0

    # Extract the error
    error = sequence[pos:]
    if error:
        result.append(error)

    #count the number of shots
    rallyCount = len(result) - 1

    return f'{rallyCount}: ' + ' -> '.join(result)

In [12]:
charting['1st_test'] = charting['1st'].apply(parse_sequence)

charting['2nd'] = charting['2nd'].astype(str)
charting['2nd_test'] = charting['2nd'].apply(parse_sequence)

In [13]:
charting

,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Gm#,TbSet,TB?,...,isUnret,isRallyWinner,isForced,isUnforced,isDouble,PtWinner,isSvrWinner,rallyCount,1st_test,2nd_test
0,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,1,0,0,0,0,0-0,1 (1),1,0,...,False,False,True,False,False,1,1,1.0,2: 6 -> f2n -> #,0: nan
1,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,2,0,0,0,0,15-0,1 (2),1,0,...,False,True,False,False,False,2,0,8.0,0: 4n,8: 6 -> b19 -> f1 -> b2 -> s1 -> f3 -> f2 -> j...
2,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,3,0,0,0,0,15-15,1 (3),1,0,...,False,True,False,False,False,2,0,4.0,0: 4d,4: 4 -> b28 -> f2 -> o1 -> *
3,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,4,0,0,0,0,15-30,1 (4),1,0,...,False,True,False,False,False,1,1,3.0,3: 6 -> s28 -> f3 -> *,0: nan
4,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,5,0,0,0,0,30-30,1 (5),1,0,...,False,True,False,False,False,1,1,3.0,3: 4 -> b37 -> b3 -> *,0: nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246625,20200103-M-ATP_Cup-RR-Alex_De_Minaur-Alexander...,203,1,1,5,2,15-0,31 (2),1,0,...,False,False,False,True,False,1,1,11.0,0: 6w,12: 5 -> b39 -> b2 -> f1 -> f1 -> f2 -> b2 -> ...
246626,20200103-M-ATP_Cup-RR-Alex_De_Minaur-Alexander...,204,1,1,5,2,30-0,31 (3),1,0,...,False,True,False,False,False,1,1,9.0,0: 4n,9: 4 -> f28 -> f3 -> b2 -> f1 -> f2 -> f1 -> r...
246627,20200103-M-ATP_Cup-RR-Alex_De_Minaur-Alexander...,205,1,1,5,2,40-0,31 (4),1,0,...,False,False,False,True,False,2,0,10.0,11: 6 -> f28 -> b2 -> b3 -> b2 -> f1 -> f1 -> ...,0: nan
246628,20200103-M-ATP_Cup-RR-Alex_De_Minaur-Alexander...,206,1,1,5,2,40-15,31 (5),1,0,...,False,False,False,True,False,2,0,6.0,7: 5 -> b37 -> b2 -> b1 -> f2 -> f2 -> b2d -> @,0: nan


In [14]:
def get_rally_count(row):
    if row['1stIn'] == 1:
        if row['1st_test'][-1] != '*':
            return int(row['1st_test'].split(':')[0]) - 1
        else:
            return int(row['1st_test'].split(':')[0])
    else:
        if row['2nd_test'][-1] != '*':
            return int(row['2nd_test'].split(':')[0]) - 1
        else:
            return int(row['2nd_test'].split(':')[0])

In [15]:
charting.loc[:, 'rallyCountVerification'] = charting.apply(get_rally_count, axis=1)
charting.loc[:, 'rallyCountVerification'] = charting['rallyCountVerification'].apply(lambda x: x if x != -1 else 0)

In [16]:
charting = charting.dropna(subset=['rallyCount'])
charting.loc[:,'rallyCount'] = charting['rallyCount'].astype(int)

C:\Users\agusd\AppData\Local\Temp\ipykernel_428\1235348759.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charting.loc[:,'rallyCount'] = charting['rallyCount'].astype(int)
C:\Users\agusd\AppData\Local\Temp\ipykernel_428\1235348759.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  charting.loc[:,'rallyCount'] = charting['rallyCount'].astype(int)


In [17]:
#De esta manera se puede ver cuantos valores son diferentes, comprobando que la diferencia se da por errores de data entry
differences = charting[charting['rallyCountVerification'].ne(charting['rallyCount'])]
differences.head()

,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Gm#,TbSet,TB?,...,isRallyWinner,isForced,isUnforced,isDouble,PtWinner,isSvrWinner,rallyCount,1st_test,2nd_test,rallyCountVerification
15,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,16,0,0,2,1,15-0,4 (2),1,0,...,False,True,False,False,1,0,4,4: 4 -> b37 -> h1 -> f1 -> z#,0: nan,3
91,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,92,1,0,4,2,30-40,16 (6),1,0,...,False,True,False,False,2,1,3,3: 4 -> b38 -> f1 -> f#,0: nan,2
119,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,120,2,0,2,0,0-0,20 (1),1,0,...,False,True,False,False,2,1,3,0: 4w,3: 5 -> b29 -> f1 -> f#,2
125,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,126,2,0,2,1,15-0,21 (2),1,0,...,False,True,False,False,2,0,22,0: 4n,22: 6 -> b28 -> f2 -> b2 -> b2 -> b1 -> f1 -> ...,21
133,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,134,2,0,3,1,0-15,22 (2),1,0,...,False,False,False,False,2,1,1,1: 5 -> #,0: nan,0


In [18]:
charting.loc[:,'1st_test'] = charting['1st_test'].str.split(':').str[1]
charting.loc[:,'2nd_test'] = charting['2nd_test'].str.split(':').str[1]

C:\Users\agusd\AppData\Local\Temp\ipykernel_428\4105296349.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charting.loc[:,'1st_test'] = charting['1st_test'].str.split(':').str[1]
C:\Users\agusd\AppData\Local\Temp\ipykernel_428\4105296349.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charting.loc[:,'2nd_test'] = charting['2nd_test'].str.split(':').str[1]


In [19]:
charting_limpio = charting[charting['rallyCountVerification'].eq(charting['rallyCount'])]
charting_limpio

,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Gm#,TbSet,TB?,...,isRallyWinner,isForced,isUnforced,isDouble,PtWinner,isSvrWinner,rallyCount,1st_test,2nd_test,rallyCountVerification
0,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,1,0,0,0,0,0-0,1 (1),1,0,...,False,True,False,False,1,1,1,6 -> f2n -> #,nan,1
1,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,2,0,0,0,0,15-0,1 (2),1,0,...,True,False,False,False,2,0,8,4n,6 -> b19 -> f1 -> b2 -> s1 -> f3 -> f2 -> j2 ...,8
2,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,3,0,0,0,0,15-15,1 (3),1,0,...,True,False,False,False,2,0,4,4d,4 -> b28 -> f2 -> o1 -> *,4
3,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,4,0,0,0,0,15-30,1 (4),1,0,...,True,False,False,False,1,1,3,6 -> s28 -> f3 -> *,nan,3
4,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,5,0,0,0,0,30-30,1 (5),1,0,...,True,False,False,False,1,1,3,4 -> b37 -> b3 -> *,nan,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246625,20200103-M-ATP_Cup-RR-Alex_De_Minaur-Alexander...,203,1,1,5,2,15-0,31 (2),1,0,...,False,False,True,False,1,1,11,6w,5 -> b39 -> b2 -> f1 -> f1 -> f2 -> b2 -> b1 ...,11
246626,20200103-M-ATP_Cup-RR-Alex_De_Minaur-Alexander...,204,1,1,5,2,30-0,31 (3),1,0,...,True,False,False,False,1,1,9,4n,4 -> f28 -> f3 -> b2 -> f1 -> f2 -> f1 -> r2 ...,9
246627,20200103-M-ATP_Cup-RR-Alex_De_Minaur-Alexander...,205,1,1,5,2,40-0,31 (4),1,0,...,False,False,True,False,2,0,10,6 -> f28 -> b2 -> b3 -> b2 -> f1 -> f1 -> f3 ...,nan,10
246628,20200103-M-ATP_Cup-RR-Alex_De_Minaur-Alexander...,206,1,1,5,2,40-15,31 (5),1,0,...,False,False,True,False,2,0,6,5 -> b37 -> b2 -> b1 -> f2 -> f2 -> b2d -> @,nan,6


In [20]:
print(charting_limpio.shape)
print(charting.shape)

(224495, 33)
(246629, 33)
